In [40]:
# !pip install tokenizers
# !pip install transformers
# !pip install torch
# !pip install seqeval
# !pip install arabert

In [2]:
# https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Custom_Named_Entity_Recognition_with_BERT.ipynb
# https://www.kaggle.com/code/pemagrg/named-entity-recognition-using-bert

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
import pandas as pd
import nltk
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification
from sklearn.model_selection import train_test_split

# **Data preprocessing**

In [5]:
data = pd.read_csv('/content/drive/MyDrive/ner/corpus.csv', encoding = 'UTF-8')

# Inspect the column names to get the label names
label_names = data.columns.tolist()
print(label_names)

['Word', 'Acutal', 'IOB', 'ner_tag', 'Sentence #']


<ipython-input-5-a1d6aed63b51>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/ner/corpus.csv', encoding = 'UTF-8')


In [6]:
data.head()

,Word,Acutal,IOB,ner_tag,Sentence #
0,الجامع,Book,B-Book,15,1
1,المسند,Book,I-Book,16,1
2,الصحيح,Book,I-Book,16,1
3,المختصر,Book,I-Book,16,1
4,من,Book,I-Book,16,1


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
data.count()

Word          258241
Acutal        258241
IOB           258241
ner_tag       258241
Sentence #    258241
dtype: int64

In [9]:
# different NER tags, and their frequency:
print("Number of tags: {}".format(len(data.IOB.unique())))
frequencies = data.IOB.value_counts()
frequencies

Number of tags: 40


O            186040
I-Pers        20916
B-Pers        18243
B-Allah        7706
B-Number       7424
I-Number       6283
B-Prophet      4490
I-Prophet      2012
B-Loc          1189
B-NatOb         628
B-Clan          449
I-Clan          318
B-Date          301
I-Date          295
B-Para          287
B-Hell          237
B-Rlig          182
I-Loc           160
B-Crime         151
B-Meas          130
B-Book          123
B-Mon           121
I-Allah         105
B-Time           64
B-Month          63
I-Crime          61
I-Book           60
I-NatOb          42
I-Time           38
B-Day            31
I-Mon            18
I-Meas           17
B-Sect           15
I-Month          14
I-Hell            8
I-Para            7
I-Org             6
B-Org             3
I-Sect            2
I-Rlig            2
Name: IOB, dtype: int64

In [10]:
# Let's print actual NER tags by frequency (highest to lowest):
tags = {}
for tag, count in zip(frequencies.index, frequencies):
    if tag != "O":
        if tag[2:8] not in tags.keys():
            tags[tag[2:8]] = count
        else:
            tags[tag[2:8]] += count
    continue

print(sorted(tags.items(), key=lambda x: x[1], reverse=True))

[('Pers', 39159), ('Number', 13707), ('Allah', 7811), ('Prophe', 6502), ('Loc', 1349), ('Clan', 767), ('NatOb', 670), ('Date', 596), ('Para', 294), ('Hell', 245), ('Crime', 212), ('Rlig', 184), ('Book', 183), ('Meas', 147), ('Mon', 139), ('Time', 102), ('Month', 77), ('Day', 31), ('Sect', 17), ('Org', 9)]


In [11]:
# Let's print NER tags with IOB by frequency (highest to lowest) based on the IOB column:
tags = {}
for iob, count in zip(data['IOB'].unique(), data['IOB'].value_counts()):
    if iob != "O":
        if iob[0:5] not in tags.keys():
            tags[iob[0:7]] = count
        else:
            tags[iob[0:7]] += count

print(sorted(tags.items(), key=lambda x: x[1], reverse=True))

[('B-Book', 186040), ('I-Book', 20916), ('B-Pers', 7706), ('I-Pers', 7424), ('B-Org', 6283), ('I-Org', 4490), ('B-Date', 2012), ('B-Numbe', 1189), ('B-Allah', 628), ('I-Allah', 449), ('I-Date', 318), ('B-Meas', 301), ('I-Meas', 295), ('B-Proph', 287), ('I-Proph', 237), ('B-Rlig', 182), ('I-Numbe', 160), ('B-Sect', 151), ('I-Sect', 130), ('I-Rlig', 123), ('B-Loc', 121), ('I-Loc', 105), ('B-Month', 64), ('B-NatOb', 63), ('B-Clan', 61), ('I-Clan', 60), ('I-NatOb', 42), ('B-Crime', 38), ('I-Crime', 31), ('B-Time', 18), ('I-Time', 17), ('B-Hell', 15), ('B-Para', 14), ('I-Para', 8), ('I-Hell', 7), ('B-Day', 6), ('I-Month', 3), ('B-Mon', 2), ('I-Mon', 2)]


In [12]:
# NEEDED: person(Pers),God(Allah),prophet  (Prophet), location (Loc), clan (Clan), date (Date), natural object (NatOb) and other (O)
# named entities since the rest are insuf cient to train the model.
entities_to_remove = ["B-Para", "I-Para", "B-Hell", "I-Hell", "B-Crime", "I-Crime", "B-Rlig", "I-Rlig", "B-Book", "I-Book", "B-Meas", "I-Meas", "B-Mon", "I-Mon", "B-Time", "I-Time", "B-Month", "I-Month", "B-Day", "I-Day", "B-Sect", "I-Sect", "B-Org", "I-Org"]
data = data[~data.IOB.isin(entities_to_remove)]
data.head()

,Word,Acutal,IOB,ner_tag,Sentence #
16,المؤلف,O,O,0,1
17,محمد,Pers,B-Pers,5,1
18,بن,Pers,I-Pers,6,1
19,إسماعيل,Pers,I-Pers,6,1
20,أبو,Pers,I-Pers,6,1


Now, we have to ask ourself the question: what is a training example in the case of NER, which is provided in a single forward pass? A training example is typically a sentence, with corresponding IOB tags. Let's group the words and corresponding tags by sentence:

In [13]:
# pandas has a very handy "forward fill" function to fill missing values based on the last upper non-nan value
data = data.fillna(method='ffill')
data.head()

,Word,Acutal,IOB,ner_tag,Sentence #
16,المؤلف,O,O,0,1
17,محمد,Pers,B-Pers,5,1
18,بن,Pers,I-Pers,6,1
19,إسماعيل,Pers,I-Pers,6,1
20,أبو,Pers,I-Pers,6,1


In [14]:
# let's create a new column called "sentence" which groups the words by sentence
# data['sentence'] = data[['Sentence #','Word','IOB']].groupby(['Sentence #'])['Word'].transform(lambda x: ' '.join(x))
data['sentence'] = data[['Sentence #','Word','IOB']].groupby(['Sentence #'])['Word'].transform(lambda x: ' '.join(str(word) for word in x))
# let's also create a new column called "word_labels" which groups the tags by sentence
data['word_labels'] = data[['Sentence #','Word','IOB']].groupby(['Sentence #'])['IOB'].transform(lambda x: ','.join(x))
data.head()

,Word,Acutal,IOB,ner_tag,Sentence #,sentence,word_labels
16,المؤلف,O,O,0,1,المؤلف محمد بن إسماعيل أبو عبد الله البخاري ال...,"O,B-Pers,I-Pers,I-Pers,I-Pers,I-Pers,I-Pers,I-..."
17,محمد,Pers,B-Pers,5,1,المؤلف محمد بن إسماعيل أبو عبد الله البخاري ال...,"O,B-Pers,I-Pers,I-Pers,I-Pers,I-Pers,I-Pers,I-..."
18,بن,Pers,I-Pers,6,1,المؤلف محمد بن إسماعيل أبو عبد الله البخاري ال...,"O,B-Pers,I-Pers,I-Pers,I-Pers,I-Pers,I-Pers,I-..."
19,إسماعيل,Pers,I-Pers,6,1,المؤلف محمد بن إسماعيل أبو عبد الله البخاري ال...,"O,B-Pers,I-Pers,I-Pers,I-Pers,I-Pers,I-Pers,I-..."
20,أبو,Pers,I-Pers,6,1,المؤلف محمد بن إسماعيل أبو عبد الله البخاري ال...,"O,B-Pers,I-Pers,I-Pers,I-Pers,I-Pers,I-Pers,I-..."


Let's have a look at the different NER tags.

We create 2 dictionaries: one that maps individual tags to indices, and one that maps indices to their individual tags. This is necessary in order to create the labels (as computers work with numbers = indices, rather than words = tags) - see further in this notebook.

In [15]:
label2id = {k: v for v, k in enumerate(data.IOB.unique())}
id2label = {v: k for v, k in enumerate(data.IOB.unique())}
label2id

{'O': 0,
 'B-Pers': 1,
 'I-Pers': 2,
 'B-Date': 3,
 'B-Number': 4,
 'B-Allah': 5,
 'I-Allah': 6,
 'I-Date': 7,
 'B-Prophet': 8,
 'I-Prophet': 9,
 'I-Number': 10,
 'B-Loc': 11,
 'I-Loc': 12,
 'B-NatOb': 13,
 'B-Clan': 14,
 'I-Clan': 15,
 'I-NatOb': 16}

As we can see, there are now only 10 different tags.

Let's only keep the "sentence" and "word_labels" columns, and drop duplicates:

In [16]:
data = data[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
data.head()

,sentence,word_labels
0,المؤلف محمد بن إسماعيل أبو عبد الله البخاري ال...,"O,B-Pers,I-Pers,I-Pers,I-Pers,I-Pers,I-Pers,I-..."
1,بن ناصر الناصر الناشر مصورة عن السلطانية بإضاف...,"I-Pers,I-Pers,I-Pers,O,O,O,O,O,O,O,B-Pers,I-Pe..."
2,وهو ضمن خدمة التخريج ومتن مرتبط بشرحه مع الكتا...,"O,O,O,O,O,O,O,O,O,O,O,O,B-Pers,I-Pers,I-Pers,O..."
3,في ط البغا يليه تعليقه ثم أطرافه مقدمة د مصطفى...,"O,O,B-Pers,O,O,O,O,O,O,B-Pers,I-Pers,O,B-Allah..."
4,والسلام على سيدنا محمد بن عبد الله الذي أرسله ...,"O,O,O,B-Pers,I-Pers,I-Pers,I-Pers,O,O,B-Allah,..."


In [17]:
len(data)

9223

In [18]:
data.iloc[4].sentence

'والسلام على سيدنا محمد بن عبد الله الذي أرسله الله تعالى رحمة للناس وآتاه الحكمة وجوامع الكلم وعلمه ما لم يكن يعلم وكان فضل الله عليه عظيما وعلى'

In [19]:
data.iloc[4].word_labels

'O,O,O,B-Pers,I-Pers,I-Pers,I-Pers,O,O,B-Allah,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-Allah,O,O,O'

# **Preparing the dataset and dataloader**

Now that our data is preprocessed, we can turn it into PyTorch tensors such that we can provide it to the model. Let's start by defining some key variables that will be used later on in the training/evaluation process:

In [43]:
MAX_LEN = 54
TRAIN_BATCH_SIZE = 32
TEST_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 10
LEARNING_RATE = 1e-04
MAX_GRAD_NORM = 10

from arabert.preprocess import ArabertPreprocessor

model_name="bert-base-arabertv01"
tokenizer = ArabertPreprocessor(model_name=model_name)

In [21]:
data.head()

,sentence,word_labels
0,المؤلف محمد بن إسماعيل أبو عبد الله البخاري ال...,"O,B-Pers,I-Pers,I-Pers,I-Pers,I-Pers,I-Pers,I-..."
1,بن ناصر الناصر الناشر مصورة عن السلطانية بإضاف...,"I-Pers,I-Pers,I-Pers,O,O,O,O,O,O,O,B-Pers,I-Pe..."
2,وهو ضمن خدمة التخريج ومتن مرتبط بشرحه مع الكتا...,"O,O,O,O,O,O,O,O,O,O,O,O,B-Pers,I-Pers,I-Pers,O..."
3,في ط البغا يليه تعليقه ثم أطرافه مقدمة د مصطفى...,"O,O,B-Pers,O,O,O,O,O,O,B-Pers,I-Pers,O,B-Allah..."
4,والسلام على سيدنا محمد بن عبد الله الذي أرسله ...,"O,O,O,B-Pers,I-Pers,I-Pers,I-Pers,O,O,B-Allah,..."


#### **Sub word tokenization**

In [22]:
# import pandas as pd
# from transformers import BertTokenizer

# def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
#     tokenized_sentence = []
#     labels = []

#     sentence = sentence.strip()

#     for word, label in zip(sentence.split(), text_labels.split(",")):

#         # Tokenize the word
#         tokenized_word = tokenizer.tokenize(word)

#         # If the word is tokenized into multiple subwords
#         if len(tokenized_word) > 1:
#             # Assign the label of the original word to all subwords
#             tokenized_sentence.extend(tokenized_word)
#             labels.extend([label] * len(tokenized_word))
#         else:
#             # Add the tokenized word to the final tokenized word list
#             tokenized_sentence.append(tokenized_word[0])
#             # Add the label to the new list of labels
#             labels.append(label)

#     return tokenized_sentence, labels

# # Load the tokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# # Read the corpus.csv file
# data = pd.read_csv("corpus.csv")

# # Drop rows with NaN values
# data.dropna(subset=["Word", "IOB"], inplace=True)

# # Tokenize each sentence and preserve labels
# tokenized_sentences = []
# tokenized_labels = []

# for i, row in data.iterrows():
#     sentence = row["Word"]
#     text_labels = row["IOB"]
#     tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, text_labels, tokenizer)
#     tokenized_sentences.append(tokenized_sentence)
#     tokenized_labels.append(labels)

# # Print tokenized sentences and labels
# for tokens, labels in zip(tokenized_sentences, tokenized_labels):
#     print("Tokenized Sentence:", tokens)
#     print("Labels:", labels)
#     print()


In [23]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

Next, we define a regular PyTorch dataset class (which transforms examples of a dataframe to PyTorch tensors). Here, each sentence gets tokenized, the special tokens that BERT expects are added, the tokens are padded or truncated based on the max length of the model, the attention mask is created and the labels are created based on the dictionary which we defined above.

In [24]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.sentence[index]
        word_labels = self.data.word_labels[index]
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)

        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]

        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return self.len

Now, based on the class we defined above, we can create 3 datasets, one for training, one for testing and one for validation. Let's use a 80/10/10 split:

In [25]:
# Define the train-test-validation split ratios
train_ratio = 0.8
test_ratio = 0.1
val_ratio = 0.1

# Split the dataset into train and temp sets
train_data, temp_data = train_test_split(data, train_size=train_ratio, random_state=200)

# Split the temp set into test and validation sets
test_data, val_data = train_test_split(temp_data, train_size=test_ratio / (test_ratio + val_ratio), random_state=200)

# Reset the indices of the datasets
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))
print("VALIDATION Dataset: {}".format(val_data.shape))

# Now you can create your datasets using the train, test, and validation data
training_set = dataset(train_data, tokenizer, MAX_LEN)
testing_set = dataset(test_data, tokenizer, MAX_LEN)
validation_set = dataset(val_data, tokenizer, MAX_LEN)

FULL Dataset: (9223, 2)
TRAIN Dataset: (7378, 2)
TEST Dataset: (922, 2)
VALIDATION Dataset: (923, 2)


In [26]:
training_set[0]

{'ids': tensor([  101,  1270, 15915,  1300, 17149, 15394,  1270, 23673, 14498, 14157,
         22192,  1270, 22192, 29836, 25573, 23673, 14157, 22192,  1298, 29824,
         29816, 14498, 14157, 22192,  1291, 29834, 25573, 23673,  1294, 14157,
         22192,  1280, 29824, 29836, 23673,  1270, 23673, 23673, 14157,  1284,
         23673, 29837,  1270, 23673, 23673, 14157,  1288, 23673, 14498, 14157,
          1298, 29824, 23673, 22192]),
 'mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]),
 'targets': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 8, 8, 8, 8, 9, 9, 9, 9, 0, 0, 0, 5, 5, 5, 5, 0, 0,
         0, 0, 0, 0, 0, 0])}

In [27]:
training_set[0]["ids"]

tensor([  101,  1270, 15915,  1300, 17149, 15394,  1270, 23673, 14498, 14157,
        22192,  1270, 22192, 29836, 25573, 23673, 14157, 22192,  1298, 29824,
        29816, 14498, 14157, 22192,  1291, 29834, 25573, 23673,  1294, 14157,
        22192,  1280, 29824, 29836, 23673,  1270, 23673, 23673, 14157,  1284,
        23673, 29837,  1270, 23673, 23673, 14157,  1288, 23673, 14498, 14157,
         1298, 29824, 23673, 22192])

In [28]:
# print the first 50 tokens and corresponding labels
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["ids"][:50]), training_set[0]["targets"][:50]):
  print('{0:10}  {1}'.format(token, id2label[label.item()]))

[CLS]       O
ا           O
##ن         O
ي           O
##ر         O
##د         O
ا           O
##ل         O
##ي         O
##ه         O
##م         O
ا           O
##م         O
##و         O
##ا         O
##ل         O
##ه         O
##م         O
و           O
##س         O
##ب         O
##ي         O
##ه         O
##م         O
ف           O
##ق         O
##ا         O
##ل         O
ل           O
##ه         O
##م         O
ر           B-Prophet
##س         B-Prophet
##و         B-Prophet
##ل         B-Prophet
ا           I-Prophet
##ل         I-Prophet
##ل         I-Prophet
##ه         I-Prophet
ص           O
##ل         O
##ى         O
ا           B-Allah
##ل         B-Allah
##ل         B-Allah
##ه         B-Allah
ع           O
##ل         O
##ي         O
##ه         O


Now, let's define the corresponding PyTorch dataloaders:

In [29]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

valid_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)
validation_loader = DataLoader(validation_set, **valid_params)

# **Defining the model**
Here we define the model, BertForTokenClassification, and load it with the pretrained weights of "bert-base-uncased". The only thing we need to additionally specify is the number of labels (as this will determine the architecture of the classification head).

Note that only the base layers are initialized with the pretrained weights. The token classification head of top has just randomly initialized weights, which we will train, together with the pretrained weights, using our labelled dataset. This is also printed as a warning when you run the code cell below.

Then, we move the model to the GPU.

In [44]:
model = BertForTokenClassification.from_pretrained(model_name,
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)
device = 'cuda'
model.to(device)

OSError: bert-base-arabertv01 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

# **Training the model**
Before training the model, let's perform a sanity check, which I learned thanks to Andrej Karpathy's wonderful cs231n course at Stanford (see also his blog post about debugging neural networks). The initial loss of your model should be close to -ln(1/number of classes) = -ln(1/17) = 2.83.

Why? Because we are using cross entropy loss. The cross entropy loss is defined as -ln(probability score of the model for the correct class). In the beginning, the weights are random, so the probability distribution for all of the classes for a given token will be uniform, meaning that the probability for the correct class will be near 1/17. The loss for a given token will thus be -ln(1/17). As PyTorch's CrossEntropyLoss (which is used by BertForTokenClassification) uses mean reduction by default, it will compute the mean loss for each of the tokens in the sequence (in other words, for all of the 512 tokens). The mean of 512 times -log(1/17) is, you guessed it, -log(1/17).

Let's verify this:

In [ ]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

This looks good. Let's also verify that the logits of the neural network have a shape of (batch_size, sequence_length, num_labels):

In [ ]:
tr_logits = outputs[1]
tr_logits.shape

Next, we define the optimizer. Here, we are just going to use Adam with a default learning rate. One can also decide to use more advanced ones such as AdamW (Adam with weight decay fix), which is included in the Transformers repository, and a learning rate scheduler, but we are not going to do that here.

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

Now let's define a regular PyTorch training function. It is partly based on a really good repository about multilingual NER.

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_preds.extend(predictions)
        tr_labels.extend(targets)

        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

And let's train the model!

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

# **Evaluating the model**
Now that we've trained our model, we can evaluate its performance on the held-out test set (which is 20% of the data). Note that here, no gradient updates are performed, the model just outputs its logits.

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(targets)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [ ]:
labels, predictions = valid(model, testing_loader)

In [ ]:
labels, predictions = valid(model, validation_loader)

However, the accuracy metric is misleading, as a lot of labels are "outside" (O), even after omitting predictions on the [PAD] tokens. What is important is looking at the precision, recall and f1-score of the individual tags. For this, we use the seqeval Python library:

In [ ]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

In [ ]:
sentence = "انا اسمي موندلي"

inputs = tokenizer(sentence, padding='max_length', truncation=True, max_length=MAX_LEN, return_tensors="pt")

# move to gpu
ids = inputs["input_ids"].to(device)
mask = inputs["attention_mask"].to(device)
# forward pass
outputs = model(ids, mask)
logits = outputs[0]

active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
token_predictions = [id2label[i] for i in flattened_predictions.cpu().numpy()]
wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

word_level_predictions = []
for pair in wp_preds:
  if (pair[0].startswith(" ##")) or (pair[0] in ['[CLS]', '[SEP]', '[PAD]']):
    # skip prediction
    continue
  else:
    word_level_predictions.append(pair[1])

# we join tokens, if they are not special ones
str_rep = " ".join([t[0] for t in wp_preds if t[0] not in ['[CLS]', '[SEP]', '[PAD]']]).replace(" ##", "")
print(str_rep)
print(word_level_predictions)